In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("what-makes-a-great-software-engineer.pdf")
data = loader.load()
data

[Document(metadata={'source': 'what-makes-a-great-software-engineer.pdf', 'page': 0}, page_content=' \nWhat Makes A Great Software Engineer? \n      \n \nMicrosoft+ \nSeattle, WA \n{pal,jiaminz}@microsoft.com \n \nThe Information School*  \nUniversity of Washington  \najko@uw.edu\nAbstract— Good software engineers are essential to the creation \nof good software. However, most of what we know about software-\nengineering expertise are vague stereotypes, such as ‘excellent \ncommunicators’ and ‘great teammates’. The lack of specificity in \nour understanding hinders researchers from reasoning about \nthem, employers from identifying them, and young engineers from \nbecoming them. Our understanding also lacks breadth:  what are \nall the distinguishing attributes of great engineers (technical \nexpertise and beyond)? We took a first step in addressing these \ngaps by interviewing 59 experienced engineers across 13 divisions \nat Microsoft, uncovering 53 attributes of great engineers. We 

In [4]:
len(data)

11

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500)
docs = text_splitter.split_documents(data)

print(type(docs))

<class 'list'>


In [6]:
import os
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.environ['GOOGLE_API_KEY'])
vector = embeddings.embed_query("hello, world!")
vector[:5]


/home/bsm/Coding/Python/RAG/langchain-gemini-sample/lang-gemini-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [7]:
# import chromadb.utils.embedding_functions as embedding_functions
# google_embed_func = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
# google_embed_func(docs)
# GoogleGenerativeAIEmbeddings
load_dotenv()
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [ ]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={"k": 10})
retrieved_docs = retriever.invoke("What are some skills to be a good software engineer?")
len(retrieved_docs)


10

In [13]:
for retrieved_doc in retrieved_docs:
    print(retrieved_doc.page_content)

the ability to come up with innovative solutions:  
TABLE 5: GREAT SOFTWARE ENGINEERS’ SOFTWARE AND DESIGNS. ATTRIBUTES DISCUSSED IN DETAIL ARE IN BOLD. 
Attribute and description Excerpt that capture interviewees’ sentiment 
Elegant—simple and intuitive (i.e. not complex) 
software/designs that others can understand. 
“They are simple... very easy to understand in a sense that it’s very simple. Doing something well 
and in a very simple way is very very hard.” -Principal Dev Lead, Bing
measurements may also form a critical foundation for managers 
to identify and cultivate talent, for novice s to improve, and for 
educators to assess learning outcomes.  
B.  For Novice Engineers  
New software engineers are often unsure of how to become 
great engineer s [1]. Our findings enumerate a set of attributes 
that they might aspire to achieve. Improvements might come 
from trainings, projects at work, mentoring, or self -adjustments
We began with open coding , identifying and assessing all 


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash', temprature=0, max_tokens=300)


In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for answering questions. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know."
    "Keep your answer precise and format your answer in bullet points."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [16]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [18]:
response = rag_chain.invoke({"input": "What are some skills to be a good software engineer?"})
print(response["answer"])

Based on the provided text, here are some attributes of great software engineers:

*   **Ability to learn on the job:** Continuously improving and learning new skills is critical.
*   **Creating elegant software/designs:**  Producing simple, intuitive, and easily understandable software.
*   **Human judgment, inspiration, and creativity:** These are considered more important than just technical knowledge.

The text also mentions that most attributes focus on *how* to achieve software engineering goals, rather than just *what* skills and knowledge are needed.  Therefore, while specific technical skills are implied, the emphasis is on the application and development of those skills.



Based on the provided text, here are some attributes of great software engineers:

*   **Ability to learn on the job:** Continuously improving and learning new skills is critical.
*   **Creating elegant software/designs:**  Producing simple, intuitive, and easily understandable software.
*   **Human judgment, inspiration, and creativity:** These are considered more important than just technical knowledge.

The text also mentions that most attributes focus on *how* to achieve software engineering goals, rather than just *what* skills and knowledge are needed.  Therefore, while specific technical skills are implied, the emphasis is on the application and development of those skills.